In [2]:
!pip install imutils
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.layers import Conv2D,Input, ZeroPadding2D
from keras.layers import BatchNormalization, Activation, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import RMSprop
from keras.models import Model, load_model, Sequential
from keras.metrics import Accuracy
from keras import utils
import imutils
import numpy as np


In [3]:
from keras.preprocessing.image import array_to_img, img_to_array,load_img
import matplotlib as mt
train_datagen = ImageDataGenerator(rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
batch_size = 10
train_data = train_datagen.flow_from_directory('data/train/',target_size=(100,100),batch_size=batch_size,class_mode='binary')


Found 1315 images belonging to 2 classes.


In [5]:
test_data = test_datagen.flow_from_directory('data/test/',target_size=(100,100),batch_size=batch_size,class_mode='binary')

Found 194 images belonging to 2 classes.


In [6]:
model = Sequential([
    Conv2D(100,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(100,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(50,activation='relu'),
    Dense(2,activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 100)       2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 52900)             0         
_________________________________________________________________
dropout (Dropout)            (None, 52900)             0         
_________________________________________________________________
dense (Dense)                (None, 50)                2

In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [8]:
history = model.fit_generator(train_data,steps_per_epoch=50//batch_size,epochs=15,validation_data=test_data,validation_steps=20//batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
5/5 [==============================] - 2s 394ms/step - loss: 1.0113 - acc: 0.4800 - val_loss: 0.6935 - val_acc: 0.3500
Epoch 2/15
5/5 [==============================] - 2s 323ms/step - loss: 0.6935 - acc: 0.4600 - val_loss: 0.6932 - val_acc: 0.6000
Epoch 3/15
5/5 [==============================] - 2s 415ms/step - loss: 0.6932 - acc: 0.3200 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 4/15
5/5 [==============================] - 1s 291ms/step - loss: 0.6932 - acc: 0.4600 - val_loss: 0.6932 - val_acc: 0.6000
Epoch 5/15
5/5 [==============================] - 2s 310ms/step - loss: 0.6932 - acc: 0.6200 - val_loss: 0.6932 - val_acc: 0.6000
Epoch 6/15
5/5 [==============================] - 2s 353ms/step - loss: 0.6932 - acc: 0.5400 - val_loss: 0.6931 - val_acc: 0.7000
Epoch 7/15
5/5 [==============================] - 2s 373ms/step - loss: 0.6931 - acc: 0.6200 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 8/15
5/5

C:\Users\LENOVO\anaconda3\lib\site-packages\PIL\Image.py:2860: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


UnknownError:  UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x00000209C8574708>
Traceback (most recent call last):

  File "C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 244, in __call__
    ret = func(*args)

  File "C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 302, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 827, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 814, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 940, in generator_fn
    yield x[i]

  File "C:\Users\LENOVO\anaconda3\lib\site-packages\keras_preprocessing\im

In [9]:
model.save('FaceMaskDetection.h5')

In [4]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import load_model
model = tf.keras.models.load_model('FaceMaskDetection.h5')

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

ValueError: in user code:

    C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1445 run